In [1]:
!pip install geopandas
!pip install tqdm

    100% |████████████████████████████████| 901kB 3.1MB/s ta 0:00:01
    100% |████████████████████████████████| 17.8MB 2.0MB/s ta 0:00:011
  Running setup.py bdist_wheel for munch ... done
  Stored in directory: /root/.cache/pip/wheels/db/bf/bc/06a3e1bfe0ab27d2e720ceb3cff3159398d92644c0cec2c125
Successfully built munch
You are using pip version 18.0, however version 19.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
    100% |████████████████████████████████| 51kB 2.1MB/s ta 0:00:011
You are using pip version 18.0, however version 19.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [1]:
import os
import boto3
import rasterio
import geopandas as gpd

from copy import copy
from shapely.geometry import box
from tqdm import tqdm_notebook as tqdm

In [2]:
s3 = boto3.client('s3')

In [3]:
bucket='naip-visualization'

In [5]:
all_directories = s3.list_objects(Bucket=bucket, Prefix='ia/2017/100cm/rgb/', Delimiter='/')

ClientError: An error occurred (AccessDenied) when calling the ListObjects operation: Access Denied

In [13]:
prefixes = [x['Prefix'] for x in all_directories['CommonPrefixes']]

In [14]:
all_tifs = []
for prefix in tqdm(prefixes):
    all_objects = s3.list_objects(Bucket=bucket, Prefix=prefix)
    dir_files = [x['Key'] for x in all_objects['Contents']]
    all_tifs += dir_files

Incorporate wind turbines dataset

In [15]:
wind_turbines = gpd.read_file('uswtdb_v1_3_20190107.geojson')

In [16]:
iowa_turbines = wind_turbines[wind_turbines.t_state == 'IA']

In [17]:
iowa_turbines_poly = copy(iowa_turbines)
iowa_turbines_poly.geometry = iowa_turbines.to_crs(epsg=26915).buffer(50)

In [18]:
for tif in tqdm(all_tifs):
    uri = 's3://{}/{}'.format(bucket, tif)
    with rasterio.open(uri) as d:
        data = d.bounds
    bbox = box(data[0], data[1], data[2], data[3])
    tmp = iowa_turbines_poly[iowa_turbines_poly.intersects(bbox)]
    tmp = tmp.to_crs({'init': 'epsg:4326'})
    if len(tmp) > 0:
        tif_local = os.path.join('output/', os.path.basename(tif))
        boto3.resource('s3').Bucket(bucket).download_file(tif, tif_local)
        outfile = tif_local.replace('.tif', '.geojson')
        tmp.to_file(driver='GeoJSON', filename=outfile)

/usr/local/lib/python3.5/dist-packages/geopandas/io/file.py:108: FionaDeprecationWarning: Use fiona.Env() instead.
  with fiona.drivers():


In [19]:
files = os.listdir('output/')

In [20]:
tifs = list(filter(lambda x: x.endswith('tif'), files))

In [21]:
geojsons = list(filter(lambda x: x.endswith('geojson'), files))

In [22]:
for g in tqdm(geojsons):
    key = 'output/{}'.format(g)
    bucketname = 'raster-vision-iowa-wind-turbines'
    filename = 'labels/{}'.format(g)
    s3.upload_file(key, bucketname, filename)

In [35]:
for t in tqdm(tifs):
    key = 'output/{}'.format(t)
    bucketname = 'raster-vision-iowa-wind-turbines'
    filename = 'images/{}'.format(t)
    s3.upload_file(key, bucketname, filename)